The presidential election dataset shows each county in the US, the candidates that won votes there and how many, and who won the county.
The 2015 county census data shows the demograpic breakup of each county, including income level, race, and other indicators.
The merged dataset will have the demographic breakup of each county and which party's candidate won that county, in order to predict whether a political party will win a county based on its demographics.

In [1]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import warnings
warnings.filterwarnings("ignore")

In [2]:
pres_data = pd.read_csv('pres16results.csv')
census_15_county = pd.read_csv('acs2015_county_data.csv')

In [3]:
# lowercase column names to match pres_data columns
census_15_county.columns = map(str.lower, census_15_county.columns)

In [4]:
# convert states in census data to state codes like in pres_data
state_codes = pd.read_csv('state_codes.csv').drop('Abbrev',1)
state_codes = state_codes.rename({'State': 'state', 'Code': 'code'}, axis = 1)

census_15_county = census_15_county.merge(state_codes, on = 'state', how = 'left').drop(columns = ['state'])\
                    .rename({'code': 'state'}, axis = 1)
census_15_county

,censusid,county,totalpop,men,women,hispanic,white,black,native,asian,...,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment,state
0,1001,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,1.0,...,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6,AL
1,1003,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,0.7,...,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5,AL
2,1005,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,0.4,...,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6,AL
3,1007,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,0.1,...,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3,AL
4,1009,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,0.1,...,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72145,Vega Baja,56858,27379,29479,96.4,3.4,0.1,0.0,0.0,...,1.3,0.3,32.0,13660,78.3,17.6,4.1,0.0,15.2,NaN
3216,72147,Vieques,9130,4585,4545,96.7,2.9,0.0,0.0,0.0,...,0.0,1.4,14.0,2860,44.5,41.6,13.6,0.3,12.2,NaN
3217,72149,Villalba,24685,12086,12599,99.7,0.0,0.0,0.0,0.0,...,0.0,3.3,26.9,6795,59.2,27.5,13.1,0.2,25.9,NaN
3218,72151,Yabucoa,36279,17648,18631,99.8,0.2,0.0,0.0,0.1,...,2.3,1.5,29.5,8083,65.1,27.6,7.3,0.0,24.3,NaN


In [5]:
pres_data['state'] = pres_data['st']
pres_data = pres_data.drop('st', 1)
pres_data

,county,fips,cand,pct_report,votes,total_votes,pct,lead,state
0,NaN,US,Donald Trump,0.9951,60350241.0,127592176.0,0.472993,Donald Trump,US
1,NaN,US,Hillary Clinton,0.9951,60981118.0,127592176.0,0.477938,Donald Trump,US
2,NaN,US,Gary Johnson,0.9951,4164589.0,127592176.0,0.032640,Donald Trump,US
3,NaN,US,Jill Stein,0.9951,1255968.0,127592176.0,0.009844,Donald Trump,US
4,NaN,US,Evan McMullin,0.9951,451636.0,127592176.0,0.003540,Donald Trump,US
...,...,...,...,...,...,...,...,...,...
18470,Yauco Municipio,72153,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18471,Midway Islands,74300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18472,St. Croix Island,78010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18473,St. John Island,78020,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Quality Issues
# pres_data has many counties NaN, need to drop because no way to figure out what those counties are
# census_data has less than 5 NaN rows, can drop because very little loss of information

In [7]:
# dropna --> can't use counties without county name or without votes recording
pres_data = pres_data.dropna()

# remove "county" from county name in pres data to match with census data
def clean_county(name):
    if 'County' in name:
        return ' '.join(name.split(' ')[:-1])
    else:
        return name

pres_data['county'] = pres_data['county'].apply(lambda x: clean_county(x))

In [8]:
# merge dataset by county to get election results and demographics for each county
# left merge to keep all counties that have election results

data = pres_data.merge(census_15_county, on = ['county', 'state'], how = 'left').dropna()
data

,county,fips,cand,pct_report,votes,total_votes,pct,lead,state,censusid,...,walk,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment
0,Los Angeles,6037,Hillary Clinton,1.0,1654626.0,2314275.0,0.714965,Hillary Clinton,CA,6037.0,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
1,Los Angeles,6037,Donald Trump,1.0,542591.0,2314275.0,0.234454,Hillary Clinton,CA,6037.0,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
2,Los Angeles,6037,Gary Johnson,1.0,56905.0,2314275.0,0.024589,Hillary Clinton,CA,6037.0,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
3,Los Angeles,6037,Jill Stein,1.0,46682.0,2314275.0,0.020171,Hillary Clinton,CA,6037.0,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
4,Los Angeles,6037,Gloria La Riva,1.0,13471.0,2314275.0,0.005821,Hillary Clinton,CA,6037.0,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17998,Kenedy,48261,Jill Stein,1.0,0.0,186.0,0.000000,Hillary Clinton,TX,48261.0,...,5.4,0.0,0.0,16.6,185.0,51.9,48.1,0.0,0.0,0.0
17999,King,48269,Donald Trump,1.0,149.0,159.0,0.937107,Donald Trump,TX,48269.0,...,7.8,0.0,3.6,11.6,166.0,72.9,21.1,6.0,0.0,5.1
18000,King,48269,Hillary Clinton,1.0,5.0,159.0,0.031447,Donald Trump,TX,48269.0,...,7.8,0.0,3.6,11.6,166.0,72.9,21.1,6.0,0.0,5.1
18001,King,48269,Gary Johnson,1.0,5.0,159.0,0.031447,Donald Trump,TX,48269.0,...,7.8,0.0,3.6,11.6,166.0,72.9,21.1,6.0,0.0,5.1


In [9]:
# reduce dataset to one row per county with whichever candidate won

# keep lead column to show winner, do not need information about number of votes each candidate got of fips
data = data.drop(columns = ['fips', 'cand', 'pct_report', 'votes', 'total_votes', 'pct', 'censusid'])
data

,county,lead,state,totalpop,men,women,hispanic,white,black,native,...,walk,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment
0,Los Angeles,Hillary Clinton,CA,10038388.0,4945351.0,5093037.0,48.2,26.9,8.0,0.2,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
1,Los Angeles,Hillary Clinton,CA,10038388.0,4945351.0,5093037.0,48.2,26.9,8.0,0.2,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
2,Los Angeles,Hillary Clinton,CA,10038388.0,4945351.0,5093037.0,48.2,26.9,8.0,0.2,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
3,Los Angeles,Hillary Clinton,CA,10038388.0,4945351.0,5093037.0,48.2,26.9,8.0,0.2,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
4,Los Angeles,Hillary Clinton,CA,10038388.0,4945351.0,5093037.0,48.2,26.9,8.0,0.2,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17998,Kenedy,Hillary Clinton,TX,565.0,295.0,270.0,66.2,33.6,0.0,0.0,...,5.4,0.0,0.0,16.6,185.0,51.9,48.1,0.0,0.0,0.0
17999,King,Donald Trump,TX,267.0,136.0,131.0,24.7,75.3,0.0,0.0,...,7.8,0.0,3.6,11.6,166.0,72.9,21.1,6.0,0.0,5.1
18000,King,Donald Trump,TX,267.0,136.0,131.0,24.7,75.3,0.0,0.0,...,7.8,0.0,3.6,11.6,166.0,72.9,21.1,6.0,0.0,5.1
18001,King,Donald Trump,TX,267.0,136.0,131.0,24.7,75.3,0.0,0.0,...,7.8,0.0,3.6,11.6,166.0,72.9,21.1,6.0,0.0,5.1


In [11]:
# many duplicates since we took out non-duplicate election columns
data = data.drop_duplicates()

In [13]:
# write dataset to csv to save

data.to_csv('cleaned_dataset.csv')

In [12]:
data

,county,lead,state,totalpop,men,women,hispanic,white,black,native,...,walk,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment
0,Los Angeles,Hillary Clinton,CA,10038388.0,4945351.0,5093037.0,48.2,26.9,8.0,0.2,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
5,Cook,Hillary Clinton,IL,5236393.0,2537245.0,2699148.0,24.7,43.1,23.7,0.1,...,4.4,2.2,4.2,32.3,2463655.0,83.9,11.5,4.5,0.1,10.7
9,Harris,Hillary Clinton,TX,4356362.0,2166727.0,2189635.0,41.6,31.7,18.5,0.2,...,1.5,2.0,3.3,28.2,2081889.0,83.4,10.1,6.3,0.1,7.5
13,Maricopa,Donald Trump,AZ,4018143.0,1986158.0,2031985.0,30.1,57.3,4.9,1.6,...,1.6,2.6,5.9,25.5,1821038.0,82.5,11.7,5.7,0.2,7.7
17,Miami-Dade,Hillary Clinton,FL,2639042.0,1280221.0,1358821.0,65.6,15.1,16.8,0.1,...,2.2,1.9,4.3,29.9,1204871.0,81.9,10.2,7.7,0.2,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,McPherson,Donald Trump,NE,433.0,223.0,210.0,0.9,97.5,0.0,0.0,...,15.7,4.6,22.6,32.0,222.0,69.4,5.9,23.9,0.9,0.9
17983,Clark,Donald Trump,ID,901.0,440.0,461.0,41.4,58.2,0.0,0.0,...,13.0,0.0,3.8,17.0,442.0,74.4,21.5,3.4,0.7,3.9
17991,Arthur,Donald Trump,NE,448.0,223.0,225.0,0.0,98.9,0.0,0.0,...,12.4,0.0,19.9,19.5,193.0,54.4,17.1,27.5,1.0,4.0
17995,Kenedy,Hillary Clinton,TX,565.0,295.0,270.0,66.2,33.6,0.0,0.0,...,5.4,0.0,0.0,16.6,185.0,51.9,48.1,0.0,0.0,0.0
